In [1]:
%pip install prettytable
%pip install metpy

  Using cached prettytable-3.8.0-py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached MetPy-1.5.1-py3-none-any.whl (400 kB)
  Using cached Pint-0.22-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# importing required library
from prettytable import PrettyTable
import pandas as pd
import xarray as xr
import os
import metpy
from metpy.units import units
import numpy as np
from pathlib import Path

In [3]:
# Get the file names
basepath = Path('/home/jovyan/Landsat_SST_algorithm')
atmpath = basepath / 'Data/ERA5_atmprofiles/Atmospheres/'
sstpath = basepath / 'Data/ERA5_atmprofiles/SSTs/'

In [31]:
def open_ERA5(monthfile,path):
    # Open all files from one month into dataframes and concatenate
    ds = xr.open_dataset(path / monthfile)
    ds = ds.drop_sel(longitude=np.delete(ds.longitude, np.arange(0, ds.longitude.size, 4)))
    df = ds.to_dataframe()
    df = df.reset_index()
    
    # print(ds.dims['longitude']*ds.dims['latitude']*ds.dims['time'])
    
    # Needs to be all levels from one profile, then next profile, then concatenate
    df = df.sort_values(by=['longitude', 'latitude','time'])
    
    return df

In [5]:
# Create MODTRAN atm correction input files

In [47]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

dir_list = os.listdir(atmpath)
sst_list = os.listdir(sstpath)

for i in months:
    
    # Prep atmospheric profiles
    # Find all files for a month
    monthfiles = [file for file in dir_list if file.endswith(f'{i}.nc')]
    print (monthfiles)
    
    dxs = []
    
    # Open each file and concatenate all files together
    for monthfile in monthfiles:
        dx = open_ERA5(monthfile,atmpath)
        dxs.append(dx)

    df = pd.concat(dxs, ignore_index=True)
    
    # Choose specific days and hours to thin the data - we chose Day 1 00h, Day 7 12h, Day 15 6h, Day 23 18h
    df = df[(df.time.dt.day==1)&(df.time.dt.hour==0)|(df.time.dt.day==7)&(df.time.dt.hour==12)|(df.time.dt.day==15)&(df.time.dt.hour==6)|(df.time.dt.day==23)&(df.time.dt.hour==18)]
    # print(f'Atm number: {df[df.level==1].shape[0]}')
    
    # Prep SSTs
    # Find all files for a month
    monthfiles = [file for file in sst_list if file.endswith(f'{i}.nc')]
    print (monthfiles)
    
    dxs = []
    
    # Open each file and concatenate all files together
    for monthfile in monthfiles:
        dx = open_ERA5(monthfile,sstpath)
        dxs.append(dx)

    sstf = pd.concat(dxs, ignore_index=True)
    
    # Choose specific days and hours to thin the data - Day 1 00h, Day 7 12h, Day 15 6h, Day 23 18h
    sstf = sstf[(sstf.time.dt.day==1)&(sstf.time.dt.hour==0)|(sstf.time.dt.day==7)&(sstf.time.dt.hour==12)|(sstf.time.dt.day==15)&(sstf.time.dt.hour==6)|(sstf.time.dt.day==23)&(sstf.time.dt.hour==18)]
    # print(f'SST number: {sstf.shape[0]}')
    
    # Remove measurements not over the ocean
    is_sst = sstf[sstf['sst'].notna()]
    
    # Convert to Celsius and remove measurements over frozen ocean
    is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15
    is_sst = is_sst[is_sst['sst'] > -1.9]
    
    # Trim atmospheric profiles based on remaining SST measurements and merge the dataframes
    good_df = pd.merge(df,is_sst,how='inner',left_on=['longitude','latitude','time'],right_on=['longitude','latitude','time'])

    print(f'Max SST: {good_df.sst.max()}, Min: {good_df.sst.min()}, Size: {is_sst.shape[0]}')
    
    # Ensure the merge produced the expected output
    if is_sst.shape[0] != good_df[good_df['level']==1].shape[0]: 
        print ('SST and atm profiles do not match') 
        continue

    # Add units to level *slow
    good_df ['hPa'] = good_df['level'].apply(lambda x: x*units.hectopascal)

    # Convert hPa to height in km *takes a long time
    good_df['ht[km]'] = good_df['hPa'].apply(metpy.calc.pressure_to_height_std)
    good_df['ht'] = good_df['ht[km]'].apply(lambda x: x.magnitude)
    
    # Prep for output and save to file
    good_df = good_df.reset_index(drop=True)
    good_df['ht'] = np.around(good_df['ht'],1)
    good_df['t'] = np.around(good_df['t'],1)
    good_df['q'] = np.around(good_df['q'],7)
     
    # Save atmopheric profiles and SSTs
    outFile = basepath / f'Data/AtmCorrection/modtran_atmprofiles_{i}_20230823.txt'
    good_df[['ht','level','t','q']].to_csv(outFile,sep='\t',index=False,header=False, encoding='ascii')
    
    outFile = basepath / f'Data/AtmCorrection/modtran_sstprofiles_{i}_20230823.txt'
    is_sst[['sst']].to_csv(outFile,sep='\t',index=False,header=False, encoding='ascii')

['era5_daily_202001.nc', 'era5_daily_202101.nc']
['era5_SST_202101.nc', 'era5_SST_202001.nc']
Max SST: 2.794342041015625, Min: -1.6900634765625, Size: 1629


/tmp/ipykernel_117/2498949025.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15


['era5_daily_202102.nc', 'era5_daily_202002.nc']
['era5_SST_202002.nc', 'era5_SST_202102.nc']
Max SST: 2.923004150390625, Min: -1.743988037109375, Size: 1627


/tmp/ipykernel_117/2498949025.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15


['era5_daily_202003.nc', 'era5_daily_202103.nc']
['era5_SST_202103.nc', 'era5_SST_202003.nc']
Max SST: 3.113189697265625, Min: -1.85552978515625, Size: 1630


...

/tmp/ipykernel_117/2498949025.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15


['era5_daily_202111.nc', 'era5_daily_202011.nc']
['era5_SST_202011.nc', 'era5_SST_202111.nc']
Max SST: 0.677398681640625, Min: -1.7630615234375, Size: 1631


/tmp/ipykernel_117/2498949025.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15


['era5_daily_202012.nc', 'era5_daily_202112.nc']
['era5_SST_202112.nc', 'era5_SST_202012.nc']
Max SST: 1.318267822265625, Min: -1.691741943359375, Size: 1629


/tmp/ipykernel_117/2498949025.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_sst.loc[:,'sst'] = is_sst['sst'] - 273.15


In [27]:
is_sst.shape[0]

1632

In [63]:
sst_list = os.listdir(sstpath)

for i in months[0:1]:
    
    # Find all files for a month
    monthfiles = [file for file in sst_list if file.endswith(f'{i}.nc')]
    print (monthfiles)
    
    dxs = []
    
    # Open each file and concatenate all files together
    for monthfile in monthfiles:
        dx = open_ERA5(monthfile,sstpath)
        dxs.append(dx)

    df = pd.concat(dxs, ignore_index=True)
    
    # Choose specific days and hours to thin the data - Day 1 00h, Day 7 12h, Day 15 6h, Day 23 18h
    df = df[(df.time.dt.day==1)&(df.time.dt.hour==0)|(df.time.dt.day==7)&(df.time.dt.hour==12)|(df.time.dt.day==15)&(df.time.dt.hour==6)|(df.time.dt.day==23)&(df.time.dt.hour==18)]
    print(df.shape[0])

#     # Add units to level *slow
#     df ['hPa'] = df['level'].apply(lambda x: x*units.hectopascal)

#     # Convert hPa to height in km *takes a long time
#     df['ht[km]'] = df['hPa'].apply(metpy.calc.pressure_to_height_std)
#     df['ht'] = df['ht[km]'].apply(lambda x: x.magnitude)

#     # Save output
#     dffile = basepath / f'Data/AtmCorrection/modtran_atmprofiles_{i}_20230823'
#     df.to_pickle(dffile)
#     pd.read_pickle(dffile)
    
#     # Prep for output and save to file
#     df = df.reset_index(drop=True)
#     df['ht'] = np.around(df['ht'],1)
#     df['t'] = np.around(df['t'],1)
#     df['q'] = np.around(df['q'],7)
    
    outFile = basepath / f'Data/AtmCorrection/modtran_sstprofiles_{i}_20230823.txt'
    df
    # df[['ht','level','t','q']].to_csv(outFile,sep='\t',index=False,header=False, encoding='ascii')


['era5_SST_202101.nc', 'era5_SST_202001.nc']
3872
3872
1936


In [67]:
is_sst = df[df['sst'].notna()]

,longitude,latitude,time,sst
64,-120.0,-73.5,2021-01-01 00:00:00,272.119904
70,-120.0,-73.5,2021-01-07 12:00:00,272.554718
73,-120.0,-73.5,2021-01-15 06:00:00,272.920135
79,-120.0,-73.5,2021-01-23 18:00:00,273.125031
80,-120.0,-73.0,2021-01-01 00:00:00,271.476288
...,...,...,...,...
7727,-100.0,-65.5,2020-01-23 18:00:00,275.079132
7728,-100.0,-65.0,2020-01-01 00:00:00,275.505402
7734,-100.0,-65.0,2020-01-07 12:00:00,275.236298
7737,-100.0,-65.0,2020-01-15 06:00:00,275.406250


In [68]:
# Open data and remove all profiles except every 4th longitude (offset between the two years) 
ds = xr.open_dataset(monthfile[0])
ds = ds.drop_sel(longitude=np.delete(ds.longitude, np.arange(0, ds.longitude.size, 4)))
ds1 = xr.open_dataset(monthfile[1])
ds1 = ds1.drop_sel(longitude=np.delete(ds1.longitude, np.arange(2, ds1.longitude.size, 4)))
print(ds.dims['longitude']*ds.dims['latitude']*ds.dims['time']+ds1.dims['longitude']*ds1.dims['latitude']*ds1.dims['time'])
ds

24552


<xarray.Dataset>
Dimensions:    (longitude: 11, latitude: 9, level: 37, time: 124)
Coordinates:
  * longitude  (longitude) float32 -120.0 -118.0 -116.0 ... -104.0 -102.0 -100.0
  * latitude   (latitude) float32 -71.3 -71.8 -72.3 -72.8 ... -74.3 -74.8 -75.3
  * level      (level) int32 1 2 3 5 7 10 20 30 ... 850 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 2020-01-01 ... 2020-01-31T18:00:00
Data variables:
    t          (time, level, latitude, longitude) float32 ...
    q          (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-03-29 00:20:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [72]:
df = ds.to_dataframe()
df = df.reset_index()
# Needs to be all levels from one profile, then next profile
df = df.sort_values(by=['longitude', 'latitude','time'])

df1 = ds1.to_dataframe()
df1 = df1.reset_index()
# Needs to be all levels from one profile, then next profile
df1 = df1.sort_values(by=['longitude', 'latitude','time'])

df = df.append(df1, ignore_index=True)
df

/tmp/ipykernel_197/3109889731.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1, ignore_index=True)


,longitude,latitude,level,time,t,q
0,-120.0,-75.300003,1,2020-01-01 00:00:00,285.299622,0.000004
1,-120.0,-75.300003,2,2020-01-01 00:00:00,283.331726,0.000004
2,-120.0,-75.300003,3,2020-01-01 00:00:00,272.387970,0.000004
3,-120.0,-75.300003,5,2020-01-01 00:00:00,263.120056,0.000003
4,-120.0,-75.300003,7,2020-01-01 00:00:00,251.301300,0.000003
...,...,...,...,...,...,...
908419,-99.0,-71.300003,900,2021-01-31 18:00:00,264.200958,0.001607
908420,-99.0,-71.300003,925,2021-01-31 18:00:00,266.266205,0.001619
908421,-99.0,-71.300003,950,2021-01-31 18:00:00,268.344391,0.001636
908422,-99.0,-71.300003,975,2021-01-31 18:00:00,269.780792,0.001754


In [74]:
# Add units to level *slow
df ['hPa'] = df['level'].apply(lambda x: x*units.hectopascal)
df

,longitude,latitude,level,time,t,q,hPa
0,-120.0,-75.300003,1,2020-01-01 00:00:00,285.299622,0.000004,1 hectopascal
1,-120.0,-75.300003,2,2020-01-01 00:00:00,283.331726,0.000004,2 hectopascal
2,-120.0,-75.300003,3,2020-01-01 00:00:00,272.387970,0.000004,3 hectopascal
3,-120.0,-75.300003,5,2020-01-01 00:00:00,263.120056,0.000003,5 hectopascal
4,-120.0,-75.300003,7,2020-01-01 00:00:00,251.301300,0.000003,7 hectopascal
...,...,...,...,...,...,...,...
908419,-99.0,-71.300003,900,2021-01-31 18:00:00,264.200958,0.001607,900 hectopascal
908420,-99.0,-71.300003,925,2021-01-31 18:00:00,266.266205,0.001619,925 hectopascal
908421,-99.0,-71.300003,950,2021-01-31 18:00:00,268.344391,0.001636,950 hectopascal
908422,-99.0,-71.300003,975,2021-01-31 18:00:00,269.780792,0.001754,975 hectopascal


In [75]:
# Convert hPa to height in km *takes a long time
df['ht[km]'] = df['hPa'].apply(metpy.calc.pressure_to_height_std)
df['ht'] = df['ht[km]'].apply(lambda x: x.magnitude)
# df.to_pickle('/home/jovyan/Data/modtran_atmprofiles_01_20230328')
df

,longitude,latitude,level,time,t,q,hPa,ht[km],ht
0,-120.0,-75.300003,1,2020-01-01 00:00:00,285.299622,0.000004,1 hectopascal,32.433259366900096 kilometer,32.433259
1,-120.0,-75.300003,2,2020-01-01 00:00:00,283.331726,0.000004,2 hectopascal,30.759332965029163 kilometer,30.759333
2,-120.0,-75.300003,3,2020-01-01 00:00:00,272.387970,0.000004,3 hectopascal,29.67279267441303 kilometer,29.672793
3,-120.0,-75.300003,5,2020-01-01 00:00:00,263.120056,0.000003,5 hectopascal,28.17902109391493 kilometer,28.179021
4,-120.0,-75.300003,7,2020-01-01 00:00:00,251.301300,0.000003,7 hectopascal,27.112745317964343 kilometer,27.112745
...,...,...,...,...,...,...,...,...,...
908419,-99.0,-71.300003,900,2021-01-31 18:00:00,264.200958,0.001607,900 hectopascal,0.9879671974242984 kilometer,0.987967
908420,-99.0,-71.300003,925,2021-01-31 18:00:00,266.266205,0.001619,925 hectopascal,0.7615554828072778 kilometer,0.761555
908421,-99.0,-71.300003,950,2021-01-31 18:00:00,268.344391,0.001636,950 hectopascal,0.5400457639046174 kilometer,0.540046
908422,-99.0,-71.300003,975,2021-01-31 18:00:00,269.780792,0.001754,975 hectopascal,0.3232070008045167 kilometer,0.323207


In [76]:
# pd.read_pickle('/home/jovyan/Data/modtran_atmprofiles_01_20230328')
df = df.reset_index(drop=True)
df['ht'] = np.around(df['ht'],1)
df['t'] = np.around(df['t'],1)
df['q'] = np.around(df['q'],7)
df

,longitude,latitude,level,time,t,q,hPa,ht[km],ht
0,-120.0,-75.300003,1,2020-01-01 00:00:00,285.299988,0.000004,1 hectopascal,32.433259366900096 kilometer,32.4
1,-120.0,-75.300003,2,2020-01-01 00:00:00,283.299988,0.000004,2 hectopascal,30.759332965029163 kilometer,30.8
2,-120.0,-75.300003,3,2020-01-01 00:00:00,272.399994,0.000004,3 hectopascal,29.67279267441303 kilometer,29.7
3,-120.0,-75.300003,5,2020-01-01 00:00:00,263.100006,0.000003,5 hectopascal,28.17902109391493 kilometer,28.2
4,-120.0,-75.300003,7,2020-01-01 00:00:00,251.300003,0.000003,7 hectopascal,27.112745317964343 kilometer,27.1
...,...,...,...,...,...,...,...,...,...
908419,-99.0,-71.300003,900,2021-01-31 18:00:00,264.200012,0.001607,900 hectopascal,0.9879671974242984 kilometer,1.0
908420,-99.0,-71.300003,925,2021-01-31 18:00:00,266.299988,0.001619,925 hectopascal,0.7615554828072778 kilometer,0.8
908421,-99.0,-71.300003,950,2021-01-31 18:00:00,268.299988,0.001636,950 hectopascal,0.5400457639046174 kilometer,0.5
908422,-99.0,-71.300003,975,2021-01-31 18:00:00,269.799988,0.001754,975 hectopascal,0.3232070008045167 kilometer,0.3


In [77]:
np.unique(df['level'],return_counts=True)

(array([   1,    2,    3,    5,    7,   10,   20,   30,   50,   70,  100,
         125,  150,  175,  200,  225,  250,  300,  350,  400,  450,  500,
         550,  600,  650,  700,  750,  775,  800,  825,  850,  875,  900,
         925,  950,  975, 1000]),
 array([24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552,
        24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552,
        24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552,
        24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552, 24552,
        24552]))

In [78]:
df[['ht','level','t','q']].to_csv('/home/jovyan/Data/modtran_atmprofiles_01_20230328.txt',sep='\t',index=False,header=False, encoding='ascii')

In [79]:
df[['ht','level','t','q']]

,ht,level,t,q
0,32.4,1,285.299988,0.000004
1,30.8,2,283.299988,0.000004
2,29.7,3,272.399994,0.000004
3,28.2,5,263.100006,0.000003
4,27.1,7,251.300003,0.000003
...,...,...,...,...
908419,1.0,900,264.200012,0.001607
908420,0.8,925,266.299988,0.001619
908421,0.5,950,268.299988,0.001636
908422,0.3,975,269.799988,0.001754


In [29]:
# np.savetxt(r'/home/jovyan/Data/modtran_AVGatmprofiles_01_20230329.txt', df[['ht','level','t','q']], fmt='%1.3f')